In [ ]:
#!pip install PyGithub
#!pip install openai
#!pip install python-dotenv

In [ ]:
import os
import openai
import requests
import subprocess
from github import Github
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
#print(os.getenv("GITHUB_REPOSITORY_URL"))

# Function to authenticate and get GitHub repository details
def get_repository_details(repo_url, github_token):
    try:
        # Authenticate with GitHub using provided token
        g = Github(github_token)
        # Get repository details
        repo = g.get_repo(repo_url)
        return repo
    except Exception as e:
        print(f"Error: {e}")

# Function to fetch Python code from a GitHub repository
def fetch_python_code_from_repository(repo, local_path):
    try:
        # Run 'git clone' command to clone the repository to the specified local path
        subprocess.run(["git", "clone", repo.clone_url, local_path], check=True)
        print("Repository cloned successfully.")

        # Fetch Python code files from the cloned repository
        python_code_files = find_python_files(local_path)

        # Read the contents of each Python code file and store them in a dictionary
        fetched_code = {}
        for file_name in python_code_files:
            file_path = os.path.join(local_path, file_name)
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                fetched_code[file_name] = file.read()

        return fetched_code
    except subprocess.CalledProcessError as e:
        print(f"Error cloning repository: {e}")
        return None

# Function to find Python files in a directory
def find_python_files(directory):
    python_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a '.py' extension
            if file.endswith(".py"):
                python_files.append(file)
    return python_files

# Function to analyze Python code using ChatGPT
def analyze_code_with_chatgpt(code_snippet, chatgpt_token):
    # Set the OpenAI API key
    openai.api_key = chatgpt_token

    # Example OpenAI Python library request
    MODEL = "gpt-3.5-turbo"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Analyze the following code and give me an optimized version with the changes you made and also include comments in the code:"},
            {"role": "assistant", "content": "Sure, please provide the code snippet."},
            {"role": "user", "content": code_snippet},
        ],
        temperature=0,
    )

    return response['choices'][0]['message']['content']

if __name__ == "__main__":
    # Replace with your GitHub repository URL and token
    github_repository_url = os.getenv("GITHUB_REPOSITORY_URL")
    #github_repository_url = "Rishtosh786/Web_Scrapping"
    github_access_token = os.getenv("GITHUB_ACCESS_TOKEN")
    
    # Replace with your ChatGPT token
    chatgpt_token = os.getenv("CHATGPT_ACCESS_TOKEN")
    
    # Fetch repository details
    repository = get_repository_details(github_repository_url, github_access_token)

    if repository:
        print(f"Repository found: {repository.full_name}")
    else:
        print("Repository not found or error occurred.")

    # Fetch Python code from the repository
    local_path = os.getenv("LOCAL_PATH")
    #local_path = "C:\\Users\\R I S H T O S H\\OneDrive\\Desktop\\XeroCode\\LocalPath"
    fetched_code = fetch_python_code_from_repository(repository, local_path)

    code_snippet = ""
    if fetched_code:
        for file_name, code in fetched_code.items():
            code_snippet = code
    else:
        print("Error fetching Python code from the repository.")

    # Analyze code using ChatGPT
    print("Analyzing code using ChatGPT...")
    analysis_result = analyze_code_with_chatgpt(code_snippet, chatgpt_token)

    # Print the result
    print("Analysis Result:")
    print(analysis_result)
